In [29]:
from gensim.test.utils import datapath

In [17]:
from gensim.test.utils import common_texts
from gensim.corpora.dictionary import Dictionary
from gensim.models.ldamulticore import LdaMulticore
from gensim.models import LdaModel
from gensim import corpora
import pandas as pd
import numpy as np
import gzip
import re
import string
from tqdm import tqdm
from time import time
from hunspell import HunSpell
from multiprocessing import pool
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from collections import Counter
from gensim.parsing.porter import PorterStemmer
import pickle
import nltk

nltk.download('stopwords')

spellchecker = HunSpell('dicts_hun/en_US.dic',
                        'dicts_hun/en_US.aff')

stemmer = PorterStemmer()

names = ['Amazon_Instant_Video', 
         'Apps_for_Android', 
         'Automotive', 
         'Baby', 
         'Beauty', 
         'Digital_Music', 
         'Grocery_and_Gourmet_Food', 
         'Health_and_Personal_Care', 
         'Home_and_Kitchen', 
         'Kindle_Store'
        ]

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

[nltk_data] Downloading package stopwords to /home/ubuntu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [18]:

ans = ''

for name in tqdm(names):

    df = getDF('data/reviews/reviews_{}.json.gz'.format(name))
    ans += df.reviewText + ' ' + df.summary + ' '

n = 50000
re_punctuation = re.compile('['+string.punctuation+']')
tokenizer = RegexpTokenizer('\w+')
stop = stopwords.words('english')
preprocessed_comments = []
for comment in tqdm(np.random.choice(ans, n)):
    comment = re_punctuation.sub(' ', comment)
    comment = tokenizer.tokenize(comment)
    comment = [x for x in comment if not any(c.isdigit() for c in x)]
    comment = [stemmer.stem(x) for x in comment if spellchecker.spell(x)]
#     comment = [ for x in comment]
    comment = [word for word in comment if word not in stop]
    preprocessed_comments.append(comment)
    
    
wordFrequency = Counter()
for comment in preprocessed_comments:
    wordFrequency.update(comment)                                  # Count overall word frequency
print('Unique Words In Comments: {}'.format(len(wordFrequency)))

minimumWordOccurrences = 5
# Remove rare words
texts = [[word for word in comment if wordFrequency[word] > minimumWordOccurrences] for comment in preprocessed_comments]

dictionary = corpora.Dictionary(texts)                             # Create word dictionary
vocabulary = [dictionary[i] for i in dictionary.keys()]
print('Documents/Comments: {}'.format(len(texts)))

corpus = [dictionary.doc2bow(doc) for doc in preprocessed_comments] # Create corpus

with open('results/corpus.pickle_', 'wb') as f:
        pickle.dump(corpus, f)


 10%|█         | 5166/50000 [00:29<04:11, 177.98it/s]


 21%|██        | 10544/50000 [01:00<03:41, 177.93it/s]


 32%|███▏      | 15861/50000 [01:30<03:09, 179.72it/s]


 42%|████▏     | 21158/50000 [02:00<02:47, 172.48it/s]


 53%|█████▎    | 26476/50000 [02:30<02:13, 176.24it/s]


...

 74%|███████▍  | 37084/50000 [03:31<01:12, 178.63it/s]


 85%|████████▍ | 42414/50000 [04:01<00:41, 184.50it/s]


 95%|█████████▌| 47746/50000 [04:31<00:13, 165.98it/s]


100%|██████████| 50000/50000 [04:44<00:00, 175.89it/s]

Unique Words In Comments: 30588
Documents/Comments: 50000


In [20]:
with open('results/dictionary.pickle_', 'wb') as f:
        pickle.dump(dictionary, f)
with open('results/vocab.pickle_', 'wb') as f:
        pickle.dump(vocabulary, f)

In [23]:
numberTopics = 20   #Number of topics

model_gensim = LdaMulticore(num_topics=numberTopics,
                        id2word=dictionary,
                        iterations=10,
                        passes=1,
                        chunksize=50,
                        eta='auto',
                        workers=14)


perp_gensim = []
times_gensim = []
i=0
max_it = 5
min_prep = np.inf
start = time()

for _ in tqdm(range(100)):
    model_gensim.update(corpus)
    tmp = np.exp(-1 * model_gensim.log_perplexity(corpus))
    perp_gensim.append(tmp)
    times_gensim.append(time() - start)
    if(tmp<min_prep):
        min_prep = tmp;
        i = 0
    else:
        i = i + 1;
        if (i==max_it):
            break                # if prep increase for max_it number it will break the update procedure 
for i, topic in enumerate(model_gensim.get_topics().argsort(axis=1)[:, -10:][:, ::-1], 1):
    print('Topic {}: {}'.format(i, ' '.join([vocabulary[id] for id in topic])))



  0%|          | 0/100 [00:00<?, ?it/s]

  1%|          | 1/100 [01:33<2:33:40, 93.13s/it]

  2%|▏         | 2/100 [03:08<2:33:09, 93.77s/it]

  3%|▎         | 3/100 [04:45<2:33:15, 94.80s/it]

  4%|▍         | 4/100 [06:20<2:31:51, 94.91s/it]

  5%|▌         | 5/100 [07:54<2:29:39, 94.52s/it]

  6%|▌         | 6/100 [09:29<2:28:11, 94.59s/it]

  7%|▋         | 7/100 [11:03<2:26:33, 94.56s/it]

  8%|▊         | 8/100 [12:38<2:25:10, 94.68s/it]

  9%|▉         | 9/100 [14:13<2:23:32, 94.65s/it]

 10%|█         | 10/100 [15:47<2:21:54, 94.61s/it]

 11%|█         | 11/100 [17:21<2:20:05, 94.45s/it]

 12%|█▏        | 12/100 [18:56<2:18:39, 94.54s/it]

 13%|█▎        | 13/100 [20:31<2:17:05, 94.55s/it]

 14%|█▍        | 14/100 [22:06<2:16:00, 94.89s/it]

 15%|█▌        | 15/100 [23:41<2:14:19, 94.82s/it]

 16%|█▌        | 16/100 [25:18<2:13:31, 95.38s/it]

 17%|█▋        | 17/100 [26:56<2:13:10, 96.27s/it]

 18%|█▊        | 18/100 [28:31<2:11:13, 96.02s/it]

 19%|█▉        | 19/100 [30:08

Topic 1: thi us food ar tast product cook like make eat
Topic 2: thi tea love like smell great wa veri good flavor
Topic 3: album thi song music sound ar wa band like record
Topic 4: thi thei ar like great good love wa veri book
Topic 5: thi us skin product oil face wa work cream great
Topic 6: coffe thi cup us water great make pot good like
Topic 7: thi great bottl wa us love work good product time
Topic 8: thi seat wa car us ar fit stroller veri back
Topic 9: hi thi stori wa charact ar ha book thei life
Topic 10: quot thi album ar hi like wa track cd song
Topic 11: thi wa take us dai time great help product ar
Topic 12: thi great love product good wa veri us price like
Topic 13: thi babi great wa love veri us good sleep like
Topic 14: thi us batteri work great unit power wa charg ar
Topic 15: game ar thei thi plai love fun great like color
Topic 16: us thi clean water diaper get ar wa thei bag
Topic 17: wa thi amazon product thei order time replac filter return
Topic 18: hair thi us 

In [37]:
model_gensim.save('results/model_genism/model_genism.model')

In [38]:
lda = LdaModel.load("results/model_genism/model_genism.model", mmap='r')

In [53]:
for i, topic in enumerate(model_gensim.get_topics().argsort(axis=1)[:, -10:][:, ::-1], 1):
    print('Topic {}: {}'.format(i, ' '.join([vocabulary[id] for id in topic])))

Topic 1: thi us food ar tast product cook like make eat
Topic 2: thi tea love like smell great wa veri good flavor
Topic 3: album thi song music sound ar wa band like record
Topic 4: thi thei ar like great good love wa veri book
Topic 5: thi us skin product oil face wa work cream great
Topic 6: coffe thi cup us water great make pot good like
Topic 7: thi great bottl wa us love work good product time
Topic 8: thi seat wa car us ar fit stroller veri back
Topic 9: hi thi stori wa charact ar ha book thei life
Topic 10: quot thi album ar hi like wa track cd song
Topic 11: thi wa take us dai time great help product ar
Topic 12: thi great love product good wa veri us price like
Topic 13: thi babi great wa love veri us good sleep like
Topic 14: thi us batteri work great unit power wa charg ar
Topic 15: game ar thei thi plai love fun great like color
Topic 16: us thi clean water diaper get ar wa thei bag
Topic 17: wa thi amazon product thei order time replac filter return
Topic 18: hair thi us 

In [61]:
lda.show_topics(num_topics=7, num_words=50, log=False, formatted=True)

[(17,
  '0.054*"hair" + 0.032*"thi" + 0.023*"us" + 0.011*"great" + 0.011*"product" + 0.011*"wa" + 0.010*"veri" + 0.009*"brush" + 0.009*"iron" + 0.009*"work" + 0.008*"get" + 0.008*"like" + 0.008*"shave" + 0.008*"time" + 0.008*"good" + 0.007*"dry" + 0.007*"shampoo" + 0.007*"love" + 0.007*"look" + 0.006*"ha" + 0.005*"dryer" + 0.005*"ar" + 0.005*"would" + 0.005*"well" + 0.005*"realli" + 0.005*"condition" + 0.005*"razor" + 0.005*"much" + 0.005*"curl" + 0.004*"make" + 0.004*"onli" + 0.004*"year" + 0.004*"doe" + 0.004*"long" + 0.004*"littl" + 0.004*"color" + 0.004*"head" + 0.004*"becaus" + 0.004*"also" + 0.004*"heat" + 0.004*"leav" + 0.004*"shaver" + 0.003*"thick" + 0.003*"need" + 0.003*"thei" + 0.003*"recommend" + 0.003*"best" + 0.003*"nice" + 0.003*"want" + 0.003*"go"'),
 (18,
  '0.034*"thi" + 0.019*"wa" + 0.018*"us" + 0.011*"get" + 0.009*"time" + 0.009*"product" + 0.008*"thei" + 0.008*"like" + 0.007*"work" + 0.007*"nail" + 0.007*"would" + 0.006*"look" + 0.006*"first" + 0.006*"ar" + 0.005*"

In [60]:
stemmer.stem('are')

'ar'